In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import csv
import time

In [ ]:
options = Options()
# options.add_argument("--headless")
path = r'D:\@ff@n\Cloud Data Engineering\Selenium Master Project\Selenium-MasterProject\chromedriver-win64\chromedriver.exe'
driver = webdriver.Chrome(path)

alldata = []

with open(r'Cryptodata.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Rank', 'Currency_Name', 'Price', 'Hour Change', 'Day Change', 'Week Change', 'Market_Capital', 'Volume', 'Circulating_Supply'])

    page = 1
    while True:
        url = f"https://coinmarketcap.com/?page={page}"
        driver.get(url)
        print(f"Scraping page: {page}")
        time.sleep(2)

        scroll_height = driver.execute_script("return document.body.scrollHeight")
        current_position = 0
        while current_position < scroll_height:
            current_position += 300  
            driver.execute_script(f"window.scrollTo(0, {current_position});")
            time.sleep(0.5) 
            scroll_height = driver.execute_script("return document.body.scrollHeight")

        time.sleep(2)  

        try:
            table = driver.find_element(By.CLASS_NAME, 'cmc-table')
            table_rows = table.find_elements(By.TAG_NAME, 'tr')

            if len(table_rows) <= 1:
                print("No more data. Reached the last page.")
                break

            for table_row in table_rows[1:]:
                table_data = table_row.find_elements(By.TAG_NAME, 'td')
                if len(table_data) < 10:
                    continue
                rank = table_data[1].text
                currency_name = table_data[2].text
                price = table_data[3].text
                hour_change = table_data[4].text
                day_change = table_data[5].text 
                week_change = table_data[6].text
                market_cap = table_data[7].text
                volume = table_data[8].text
                circulating_supply = table_data[9].text
                data = [rank, currency_name, price, hour_change, day_change, week_change, market_cap, volume, circulating_supply]
                writer.writerow(data)
                alldata.append(data)

        except Exception as e:
            print(f"Error on page {page}:", e)
            break

        page += 1

print("Total currencies scraped:", len(alldata))
driver.quit()